In [ ]:
!pip install datasets transformers[torch] sentencepiece -Uq
import random
random.seed(42)

# Introduction to Classification




### What is classification?

The task of a classification model is to classify something as a certain type of that thing. In NLP, there are many applications of this type of model, including sentiment analysis: determining whether a sentence is of positive or negative sentiment or meaning.

### Today's example
Today, we're going to create our sentiment analyizer using a classification model that will determine whether a text is of positive or negative sentiment. Specifically, this dataset is taken from tweets. Each tweet is labeled as 'Positive', 'Neutral' or 'Negative'. This is a very common way of labeling data, but we will see later in the course that how we label our data can have a profound effect on our results.

# Wrangling our data

Unfortunately, we are not able to create a model out of raw text. Instead, we have to put into a very specific form so that the software libraries we'll be using can read and process it correctly.

This dataset comes from a website called HuggingFace. This site hosts both pretrained models and datasets to train and finetune new models. We will be using [this dataset](https://huggingface.co/datasets/mteb/tweet_sentiment_extraction) which was collected by the Massive Text Embedding Benchmark. This group uses datasets like this one to evaluate how good certain models are compared to others. What we will be doing in this notebook is a key part of their evlaution process for new models.

This data comes to us as a HuggingFace Dataset. This is a specific Python-based data structure that is derived from parquet files hosted on the MTEB HuggingFace account.

In [ ]:
import datasets
from datasets import load_dataset

dataset = load_dataset('mteb/tweet_sentiment_extraction')
train = dataset['train'].to_pandas()
test = dataset['test'].to_pandas()

train['label'] = train['label'].astype('float32')
test['label'] = test['label'].astype('float32')

dataset = datasets.DatasetDict({
    'train': datasets.Dataset.from_pandas(train),
    'test': datasets.Dataset.from_pandas(test)
})

dataset

In [ ]:
# this is just a fancy spreadsheet
dataset['train'].to_pandas()

See! Just a spreadsheet! We'll talk about how I read it into our notebook in a couple classes.

Above are the sentiments in our dataset and how many there are. Each one is associated with a tweet, but they didn't just appear there magically. Someone (or more likely a group of people) had to classify these by hand. As a result, labeled data like this can be extremely valuable.

## Fine-tuning

Although, we're training our own sentiment analysis model today. We are not starting from nothing. Today we are going to do a process called 'fine-tuning.' Fine-tuning is when we take a model that someone else has trained and we train it again for a specific task. This is very common for classification tasks like this one.

Today, we'll be using Microsoft's `deberta` model.

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

`deberta` will transform the text of the tweets into mathematical representations of the text that we can use to generate our model.

What is important, however, is the the step we take before this conversion from text to numbers: **tokenization**.

### Tokenization

In [ ]:
def tok_func(x):
  return tokz(x["text"])

Tokenization is the process of breaking a sentence into the words that make it up and then associating each piece of the sentence with a number that we can refer to later.

In [ ]:
tok_ds = dataset.map(tok_func, batched=True)

For example, this is what a single post looks like now.

In [ ]:
tok_ds['train']['input_ids'][0], tok_ds['train']['text'][0]

We can make sure with the `decode` method.

In [ ]:
tokz.decode(tok_ds['train']['input_ids'][0])

### Model training

We need some way to know how well our model is doing while we train. There are many options to use but today I'll use Pearson's R. This metric will give us updates on our model while its training.

As you may have noticed above, we have two sets of data in our larger dataset: a training and test set. This training set is much larger and will be what we finetune our model on. The model will only see the test set after the model has finished looking at hte training set.

If the model looks like its going really well, and getting all of the predictions right on the training set, it's possible we have a really good model or a model that has only memorized the answers on the training set, that is a terrible model for using outside of the training set. This is called **overfitting** and is common problem in all machine learning applications.

As a result, we use the test set, a set of data that the model has never seen to give use a sense of how well the model is doing beyond the training set.

In [ ]:
import numpy as np
def corr(x,y):
  return np.corrcoef(x,y)[0][1]

def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

Now, let's get training!

In [ ]:
from transformers import TrainingArguments,Trainer

# these three hyperparameters can be changed and tweaked to return a new model.
bs = 64
epochs = 6
lr = 1e-5

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=tok_ds['train'], eval_dataset=tok_ds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train()

## Evaluation

Once training is finished, we can start predicting. These are the predictions for the evaluation set.

In [ ]:
preds = trainer.predict(trainer.eval_dataset)[1]
preds

In [ ]:
import pandas as pd
eval_df = pd.DataFrame({'text':trainer.eval_dataset['text'], 'pred':preds})
eval_df

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
p = eval_df.sample(5).apply(lambda x: pp.pprint([f"TEXT: {x['text']}", f"PREDICTION: {x['pred']}"]),axis=1)

A random sample of our evaluation data is looking good!

## Conclusion

This exercise is the most basic example of using a neural network to read sentiment from a document. There are a lot of optimizations and augmentations that can be implemented to cut down our loss even more.

I recommend the following links if you are interested in learning more:
* [Practical Deep Learning for Coders](https://course.fast.ai/)
* [The NLP HuggingFace Course](https://huggingface.co/course/chapter1/1)
* [Andrej Karpathy's Zero to Hero Series](https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)